# Jacobian Comparision between Drake and SPART 

Computing and Comparing Jacobians for Free-floating base.

Drake: https://drake.mit.edu/
SPART: https://github.com/NPS-SRL/SPART

Computing the Manipulator and Base (and combined) Jacobian for an end-effector of floating base robot. 
The state of the robot is kept same on both toolboxes. SPART computations are done in MATAB.

The SPART jacobians were previously validated in simualtion in Drake using the Generalized Jacobian Matrix (Yoshida, Umetani. 1993). However, the jacobian from the function *CalcJacobianSpatialVelocity* does not compute jacobian as expected for a free-floating base.

For more info on separating base and manipulator jacobian, see: https://spart.readthedocs.io/en/latest/Tutorial_Kinematics.html#jacobians

In [28]:
import os, sys
import time
import numpy as np
import matplotlib.pyplot as plt

if 'google.colab' in sys.modules:
  !pip install drake

from pydrake.all import MultibodyPlant, Parser, JacobianWrtVariable

# URDF Path

In [29]:
if 'google.colab' in sys.modules:
  # Check if this cell has already been run:
  if not os.path.isdir('DrakeTests/freeFloating/'):
    !git clone https://github.com/vyas-shubham/DrakeTests.git
  urdfFolderPath = 'DrakeTests/freeFloating/'
else:
  urdfFolderPath = ''

# Comment/Uncomment the lines below to select a S/C URDF
filePath = urdfFolderPath+'planarSC_no_collision.urdf'
assert os.path.isfile(filePath)

In [30]:
# Initial Conditions
q_opt = np.array([1., 0., 0., 0., 0., 0., 0., -0.72, 1.2, 0.72])
# qd_opt = np.array([0., 0.0042, 0., -0.0124, 0., -0.0052, 0.0635, -0.1353, 0.1669])

In [31]:
postContactPlant = MultibodyPlant(0.00001)
postContactPlantInstance = Parser(postContactPlant).AddAllModelsFromFile(filePath)
gravityField = postContactPlant.gravity_field()
gravityField.set_gravity_vector([0, 0, 0])


In [32]:
world_frame = postContactPlant.world_frame()
chaserSC_frame = postContactPlant.GetFrameByName("ChaserSC")
target_frame = postContactPlant.GetFrameByName("TargetSC")

postContactPlant.Finalize()

In [33]:
postContactPlant.num_positions()

10

In [34]:
postCaptureContext = postContactPlant.CreateDefaultContext()
postContactPlant.SetPositions(postCaptureContext, q_opt)
# postContactPlant.SetVelocities(postCaptureContext, qd_opt)
postContactPlant.get_actuation_input_port().FixValue(postCaptureContext, np.zeros(3))

In [35]:
postCaptureTargetJacobian = postContactPlant.CalcJacobianSpatialVelocity(context=postCaptureContext,
                                             with_respect_to=JacobianWrtVariable.kV,
                                             frame_B=target_frame,
                                             p_BP=np.zeros(3),
                                             frame_A=chaserSC_frame,
                                             frame_E=world_frame)

In [36]:
np.set_printoptions(precision=3)
print(postCaptureTargetJacobian)

[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     1.     1.     1.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     1.533  0.856  0.236]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.    -0.336 -0.929 -0.606]]


In [37]:
print("Drake Base Jacobian: ")
print(postCaptureTargetJacobian[0:6,0:6])

Drake Base Jacobian: 
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [38]:
print("Drake Manipulator Jacobian: ")
print(postCaptureTargetJacobian[0:6,-3:])

Drake Manipulator Jacobian: 
[[ 0.     0.     0.   ]
 [ 1.     1.     1.   ]
 [ 0.     0.     0.   ]
 [ 1.533  0.856  0.236]
 [ 0.     0.     0.   ]
 [-0.336 -0.929 -0.606]]


In [39]:
J_ChaserSC_TargetSC_postCap = np.loadtxt(open("J_ChaserSC_TargetSC_postCap.csv", "rb"), delimiter=",")
J_ChaserManip_TargetSC_postCap = np.loadtxt(open("J_ChaserManip_TargetSC_postCap.csv", "rb"), delimiter=",")

In [40]:
print("SPART Base Jacobian: ")
J_ChaserSC_TargetSC_postCap

SPART Base Jacobian: 


array([[ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  2.583,  0.   ,  1.   ,  0.   ,  0.   ],
       [-2.583,  0.   ,  0.336,  0.   ,  1.   ,  0.   ],
       [ 0.   , -0.336,  0.   ,  0.   ,  0.   ,  1.   ]])

In [41]:
print("SPART Manipulator Jacobian: ")
J_ChaserManip_TargetSC_postCap

SPART Manipulator Jacobian: 


array([[ 0.   ,  0.   ,  0.   ],
       [ 1.   ,  1.   ,  1.   ],
       [ 0.   ,  0.   ,  0.   ],
       [ 1.533,  0.856,  0.236],
       [ 0.   ,  0.   ,  0.   ],
       [-0.336, -0.929, -0.606]])